In [1]:
import tushare as ts
import pandas as pd
import datetime
import logging

In [2]:
#basics = ts.get_stock_basics()
#basics.to_csv('e:/quant/data/basics.csv')
def getStockBaseMarketTime(code=600030):
    basics = pd.read_csv('e:/quant/data/basics.csv')
    data = basics.loc[basics.code==code]
    timeToMarket = data['timeToMarket']
    startDate = datetime.datetime(timeToMarket/10000 ,timeToMarket%10000/100 , timeToMarket%10000%100) 
    return startDate.strftime('%Y-%m-%d')

#第一次从网络上获取然后存储在本地，后续均从本地文件获取
#内部调用函数
def getStockHistInfoInner(code=600030):
    startTime = getStockBaseMarketTime(code)
    print startTime 
    try:
        data = pd.read_csv('e:/quant/data/'+str(code)+'.csv')
        data = data.sort_values(by="date" , ascending=True)
    except IOError as e:
        dt = datetime.datetime.now()   
        data = ts.get_h_data(str(code) , start=startTime , end=dt.strftime('%Y-%m-%d'))
        data.to_csv('e:/quant/data/'+str(code)+'.csv')
        return None
    return data

#获取对应代码的股票历史行情信息
def getStockHistInfo(code=600030):
    data = getStockHistInfoInner(code)
    if data is None:
        data = getStockHistInfoInner(code)
    if data is None:
        logging.exception('the stock is still none')      
    return data
        
    
#EMA（12）= 前一日EMA（12）×11/13＋今日收盘价×2/13
#EMA（26）= 前一日EMA（26）×25/27＋今日收盘价×2/27
#DIFF=今日EMA（12）- 今日EMA（26）
#DEA（MACD）= 前一日DEA×8/10＋今日DIF×2/10 
#BAR=2×(DIFF－DEA)

def calcMacd(data=None , code=600030):
    if data is None:
        return None
    data['ema12'] = 0
    data['ema26'] = 0
    data['diff'] = 0
    data['dea'] = 0
    data['bar'] = 0
    
    
    for i in range(1,len(data),1):
        ema12 = round(round(data.iloc[i-1]['ema12'] * 11.0 / 13.0 , 4) + round(data.iloc[i]['close'] * 2.0 / 13.0 , 4) , 3)
        ema26 = round(round(data.iloc[i-1]['ema26'] * 25.0 / 27.0 , 4) + round(data.iloc[i]['close'] * 2.0 / 27.0 , 4) , 3)
        dif = round(ema12 - ema26 , 3)
        dea = round(round(data.iloc[i-1]['dea'] * 8.0 / 10.0 , 4) + round(dif * 2.0 / 10.0 , 4) , 3)
        bar = round(2 * (dif - dea) , 3)
        data.iloc[i] = data.iloc[i].set_value('ema12' , ema12) 
        data.iloc[i] = data.iloc[i].set_value('ema26' , ema26 )   
        data.iloc[i] = data.iloc[i].set_value('diff' , dif)
        data.iloc[i] = data.iloc[i].set_value('dea' , dea)
        data.iloc[i] = data.iloc[i].set_value('bar' , bar)
    data.to_csv('e:/quant/data/'+str(code)+'_macd.csv')
    return data

data = getStockHistInfo(600030)
macd = calcMacd(data , 600030)
#print data.head()
print macd.head()




2003-01-06
            date  open  high  close   low     volume      amount  ema12  \
3100  2003-01-06  1.10  1.11   1.00  0.99  194288556   998892578  0.000   
3099  2003-01-07  0.99  1.01   0.97  0.96   58977717   290622430  0.149   
3098  2003-01-08  0.96  1.06   1.06  0.96   72064078   368411113  0.289   
3097  2003-01-09  1.12  1.17   1.17  1.12  110086868   636358017  0.424   
3096  2003-01-10  1.23  1.29   1.24  1.22  206393422  1300495478  0.550   

      ema26   diff    dea    bar  
3100  0.000  0.000  0.000  0.000  
3099  0.072  0.077  0.015  0.124  
3098  0.145  0.144  0.041  0.206  
3097  0.221  0.203  0.073  0.260  
3096  0.296  0.254  0.109  0.290  
